# Aicua_SR - Symbolic Regression Training on Kaggle GPU

Train PySR models for:
- **Petal Spline V3**: 8 Control Points (heart-shaped tip)
- **Bone Rigging V5**: 7 Bones (fishbone structure)
- **Self-Awareness**: Genesis, Transformation, Composition

## Setup
1. Enable GPU: Settings → Accelerator → GPU P100/T4
2. Clone Aicua_SR repo from GitHub
3. Run training cells

In [ ]:
# Clone Aicua_SR repository
!git clone https://github.com/Aicua/Aicua_SR.git
%cd Aicua_SR

In [ ]:
# Install dependencies
!pip install pysr pandas numpy sympy scikit-learn pyyaml

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Generate datasets (V3/V5 - latest versions)
!python scripts/generate_petal_spline_v3.py
!python scripts/generate_bone_rigging_v5.py
!python scripts/generate_self_awareness_dataset.py

## Part 1: Petal Spline V3 (8 Control Points)

In [ ]:
# Load petal spline V3 dataset (8 CPs - heart-shaped tip)
import pandas as pd
import numpy as np

df = pd.read_csv('data/processed/petal_spline_v3.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nDataset statistics:")
print(f"  base_size range: {df['base_size'].min():.2f} - {df['base_size'].max():.2f}")
print(f"  Samples: {len(df)}")
df.head()

In [ ]:
# Train models for all 8 control points (V3)
from pysr import PySRRegressor

# Features (v3 naming)
features = ['base_size', 'layer_index', 'petal_index', 'opening_degree']
X = df[features].values

# All targets for 8 CPs
targets = ['cp1_x', 'cp1_y', 'cp2_x', 'cp2_y', 'cp3_x', 'cp3_y', 
           'cp4_x', 'cp4_y', 'cp5_x', 'cp5_y', 'cp6_x', 'cp6_y',
           'cp7_x', 'cp7_y', 'cp8_x', 'cp8_y', 'extrude_depth']

discovered_formulas = {}

for target in targets:
    print(f"\n{'='*60}")
    print(f"Training model for: {target}")
    print(f"{'='*60}")
    
    y = df[target].values
    
    model = PySRRegressor(
        niterations=50,  # Increase for better results
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "square"],
        populations=10,
        population_size=30,
        maxsize=20,
        parsimony=0.002,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(X, y, variable_names=features)
    
    formula = str(model.sympy())
    discovered_formulas[target] = formula
    
    print(f"✓ {target} = {formula}")

print(f"\n{'='*60}")
print("All discovered formulas for petal_spline_v3 (8 CPs):")
print(f"{'='*60}")
for target, formula in discovered_formulas.items():
    print(f"{target} = {formula}")

## Part 2: Bone Rigging V5 (7 Bones - Fishbone Structure)

In [ ]:
# Load bone_rigging_v5 (7 bones fishbone structure)
print("="*60)
print("BONE RIGGING V5 - 7 Bones Fishbone Structure")
print("="*60)

bone_df = pd.read_csv('data/processed/bone_rigging_v5.csv')
print(f"Dataset shape: {bone_df.shape}")
print(f"Features: {list(bone_df.columns[:5])}")
print(f"Targets (28): 7 bones × 4 coordinates")

# Features for bone rigging v5
bone_features = ['petal_height', 'petal_width', 'opening_degree', 'layer_index', 'curvature_intensity']
bone_X = bone_df[bone_features].values

# Key bone targets (most important ones)
bone_targets = [
    # Central spine
    'bone_root_end_y', 'bone_middle_end_y', 'bone_tip_end_y',
    # Left ribs
    'bone_left_lower_end_x', 'bone_left_lower_end_y',
    'bone_left_upper_end_x', 'bone_left_upper_end_y',
    # Right ribs
    'bone_right_lower_end_x', 'bone_right_lower_end_y',
    'bone_right_upper_end_x', 'bone_right_upper_end_y',
]

bone_formulas = {}

for target in bone_targets:
    print(f"\nTraining: {target}")
    y = bone_df[target].values
    
    model = PySRRegressor(
        niterations=30,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt"],
        populations=10,
        population_size=30,
        maxsize=15,
        parsimony=0.003,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(bone_X, y, variable_names=bone_features)
    formula = str(model.sympy())
    bone_formulas[target] = formula
    print(f"✓ {target} = {formula}")

print(f"\n{'='*60}")
print("Bone Rigging V5 Formulas (7 bones fishbone):")
print(f"{'='*60}")
for target, formula in bone_formulas.items():
    print(f"{target} = {formula}")

## Part 3: Self-Awareness Models

In [ ]:
# Load Self-Awareness Genesis dataset
print("="*60)
print("SELF-AWARENESS: GENESIS")
print("="*60)

genesis_df = pd.read_csv('data/generated/sa_genesis_dataset.csv')
print(f"Dataset shape: {genesis_df.shape}")
print(f"Columns: {list(genesis_df.columns)}")

# Genesis features and targets
genesis_features = ['layer', 'width', 'height', 'aspect_ratio', 'detail_code', 'opening_degree']
genesis_targets = ['cp_count', 'self_understanding', 'structural_confidence']

genesis_X = genesis_df[genesis_features].values

genesis_formulas = {}

for target in genesis_targets:
    print(f"\nTraining: {target}")
    y = genesis_df[target].values
    
    model = PySRRegressor(
        niterations=50,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "exp", "log"],
        populations=15,
        population_size=33,
        maxsize=20,
        parsimony=0.0032,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(genesis_X, y, variable_names=genesis_features)
    formula = str(model.sympy())
    r2 = model.score(genesis_X, y)
    genesis_formulas[target] = {'formula': formula, 'r2': r2}
    print(f"✓ {target} = {formula}")
    print(f"  R² = {r2:.4f}")

print("\nGenesis Formulas:")
for target, data in genesis_formulas.items():
    print(f"  {target}: {data['formula']} (R²={data['r2']:.4f})")

In [ ]:
# Load Self-Awareness Transformation dataset
print("="*60)
print("SELF-AWARENESS: TRANSFORMATION")
print("="*60)

transform_df = pd.read_csv('data/generated/sa_transformation_dataset.csv')
print(f"Dataset shape: {transform_df.shape}")

# Transformation features and targets
transform_features = ['layer', 'width', 'height', 'opening_degree', 'cp_count', 'self_understanding']
transform_targets = ['morph_confidence', 'avg_risk_level', 'structural_stability']

transform_X = transform_df[transform_features].values

transform_formulas = {}

for target in transform_targets:
    print(f"\nTraining: {target}")
    y = transform_df[target].values
    
    model = PySRRegressor(
        niterations=50,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "exp", "log"],
        populations=15,
        population_size=33,
        maxsize=20,
        parsimony=0.0032,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(transform_X, y, variable_names=transform_features)
    formula = str(model.sympy())
    r2 = model.score(transform_X, y)
    transform_formulas[target] = {'formula': formula, 'r2': r2}
    print(f"✓ {target} = {formula}")
    print(f"  R² = {r2:.4f}")

print("\nTransformation Formulas:")
for target, data in transform_formulas.items():
    print(f"  {target}: {data['formula']} (R²={data['r2']:.4f})")

In [ ]:
# Load Self-Awareness Composition dataset
print("="*60)
print("SELF-AWARENESS: COMPOSITION")
print("="*60)

composition_df = pd.read_csv('data/generated/sa_composition_dataset.csv')
print(f"Dataset shape: {composition_df.shape}")

# Composition features and targets
composition_features = ['layer', 'group_size', 'same_layer_count', 'adjacent_layer_count', 'self_understanding']
composition_targets = ['harmony_score', 'cooperation_confidence']

composition_X = composition_df[composition_features].values

composition_formulas = {}

for target in composition_targets:
    print(f"\nTraining: {target}")
    y = composition_df[target].values
    
    model = PySRRegressor(
        niterations=50,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "exp", "log"],
        populations=15,
        population_size=33,
        maxsize=20,
        parsimony=0.0032,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(composition_X, y, variable_names=composition_features)
    formula = str(model.sympy())
    r2 = model.score(composition_X, y)
    composition_formulas[target] = {'formula': formula, 'r2': r2}
    print(f"✓ {target} = {formula}")
    print(f"  R² = {r2:.4f}")

print("\nComposition Formulas:")
for target, data in composition_formulas.items():
    print(f"  {target}: {data['formula']} (R²={data['r2']:.4f})")

## Part 4: Save All Formulas

In [ ]:
# Save all discovered formulas to JSON
import json
import os

os.makedirs('data/generated', exist_ok=True)

all_formulas = {
    'petal_spline_v3': discovered_formulas,
    'bone_rigging_v5': bone_formulas,
    'self_awareness': {
        'genesis': genesis_formulas,
        'transformation': transform_formulas,
        'composition': composition_formulas,
    }
}

with open('data/generated/sr_discovered_formulas_v3.json', 'w') as f:
    json.dump(all_formulas, f, indent=2, default=str)

print("✓ Saved all formulas to data/generated/sr_discovered_formulas_v3.json")
print(f"\nTotal formulas discovered:")
print(f"  - Petal Spline V3: {len(discovered_formulas)} targets")
print(f"  - Bone Rigging V5: {len(bone_formulas)} targets")
print(f"  - Genesis: {len(genesis_formulas)} targets")
print(f"  - Transformation: {len(transform_formulas)} targets")
print(f"  - Composition: {len(composition_formulas)} targets")

In [ ]:
# Generate Python code from discovered formulas
def formula_to_python(formula_str, target_name, features):
    """Convert SymPy formula to Python function."""
    code = formula_str
    
    # Replace math functions
    code = code.replace('sqrt', 'math.sqrt')
    code = code.replace('exp', 'math.exp')
    code = code.replace('log', 'math.log')
    code = code.replace('sin', 'math.sin')
    code = code.replace('cos', 'math.cos')
    
    params = ', '.join(features)
    
    func_code = f"""
def compute_{target_name}({params}):
    \"\"\"SR-discovered formula for {target_name}.\"\"\"
    try:
        return {code}
    except (ValueError, ZeroDivisionError):
        return 0.0
"""
    return func_code

# Generate Python module
module_code = '''#!/usr/bin/env python3
"""Auto-generated SR formulas for Aicua_SR V3.

Generated by Kaggle GPU training.
- Petal Spline V3: 8 Control Points (heart-shaped tip)
- Bone Rigging V5: 7 Bones (fishbone structure)
- Self-Awareness: Genesis, Transformation, Composition
"""
import math

'''

# Add petal spline formulas
module_code += "# " + "="*60 + "\n"
module_code += "# PETAL SPLINE V3 (8 Control Points)\n"
module_code += "# " + "="*60 + "\n\n"

petal_features = ['base_size', 'layer_index', 'petal_index', 'opening_degree']
for target, formula in discovered_formulas.items():
    module_code += formula_to_python(formula, f'petal_{target}', petal_features)

# Add bone rigging formulas
module_code += "\n# " + "="*60 + "\n"
module_code += "# BONE RIGGING V5 (7 Bones Fishbone)\n"
module_code += "# " + "="*60 + "\n\n"

for target, formula in bone_formulas.items():
    module_code += formula_to_python(formula, f'bone_{target}', bone_features)

# Save to file
with open('data/generated/aicua_sr_formulas_v3.py', 'w') as f:
    f.write(module_code)

print("✓ Generated Python module: data/generated/aicua_sr_formulas_v3.py")

## Summary

### Trained Models:

1. **Petal Spline V3** - 17 targets (8 CPs × 2 coords + extrude_depth)
   - Heart-shaped tip with notch (CP4, CP5, CP6)
   - Undulating edges (CP2, CP3, CP7, CP8)

2. **Bone Rigging V5** - 28 targets (7 bones × 4 coords)
   - Central spine: root → middle → tip
   - Left ribs: lower, upper
   - Right ribs: lower, upper

3. **Self-Awareness**
   - Genesis: cp_count, self_understanding, structural_confidence
   - Transformation: morph_confidence, avg_risk_level, structural_stability
   - Composition: harmony_score, cooperation_confidence

### Next Steps:

1. Increase iterations for production (100-500)
2. Download `sr_discovered_formulas_v3.json`
3. Integrate formulas into Aicua_SR CLI